In [1]:
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
import os
from transformers import *
print(tf.__version__)
from sklearn.metrics import f1_score


I1015 07:36:51.441283 140024237856576 file_utils.py:39] PyTorch version 1.5.0 available.
I1015 07:36:51.443418 140024237856576 file_utils.py:55] TensorFlow version 2.0.0 available.


2.0.0


# 参数下载地址 https://huggingface.co/bert-base-chinese

In [2]:
train_left = pd.read_csv('./train/train.query.tsv',sep='\t',header=None)
train_left.columns=['id','q1']
train_right = pd.read_csv('./train/train.reply.tsv',sep='\t',header=None)
train_right.columns=['id','id_sub','q2','label']
df_train = train_left.merge(train_right, how='left')
df_train['q2'] = df_train['q2'].fillna('好的')
test_left = pd.read_csv('./test/test.query.tsv',sep='\t',header=None, encoding='gbk')
test_left.columns = ['id','q1']
test_right =  pd.read_csv('./test/test.reply.tsv',sep='\t',header=None, encoding='gbk')
test_right.columns=['id','id_sub','q2']
df_test = test_left.merge(test_right, how='left')

In [3]:
PATH = './'
BERT_PATH = './'
WEIGHT_PATH = './'
MAX_SEQUENCE_LENGTH = 100
input_categories = ['q1','q2']
output_categories = 'label'

print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

train shape = (21585, 5)
test shape = (53757, 4)


In [16]:
def _convert_to_transformer_inputs(question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy,
            #truncation=True
            )
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        question, answer, 'longest_first', max_sequence_length)
    

    
    return [input_ids_q, input_masks_q, input_segments_q]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        q, a = instance.q1, instance.q2

        ids_q, masks_q, segments_q= \
        _convert_to_transformer_inputs(q, a, tokenizer, max_sequence_length)
        
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)

    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])


def search_f1(y_true, y_pred):
    best = 0
    best_t = 0
    for i in range(30,60):
        tres = i / 100
        y_pred_bin =  (y_pred > tres).astype(int)
        score = f1_score(y_true, y_pred_bin)
        if score > best:
            best = score
            best_t = tres
    print('best', best)
    print('thres', best_t)
    return best, best_t

In [17]:
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-chinese-vocab.txt')
outputs = compute_output_arrays(df_train, output_categories)
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

I1015 07:57:39.324805 140024237856576 tokenization_utils.py:929] Model name './bert-base-chinese-vocab.txt' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased). Assuming './bert-base-chinese-vocab.txt' is a path, a model identifier, or url to a directory containing tokenizer files.
W1015 07:57:39.328269 140024237856576 tokenization_utils.py:941] Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
I1015 07:57:39.329980 1400242378565

In [24]:
def create_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig.from_pretrained('./bert-base-chinese-config.json') 
    config.output_hidden_states = False 
    bert_model = TFBertModel.from_pretrained('./bert-base-chinese-tf_model.h5', 
                                             config=config)
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalMaxPooling1D()(q_embedding)
    t = q_embedding[:,-1]
    e = q_embedding[:, 0]
    x = tf.keras.layers.Concatenate()([q, a, t, e])
    
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn], outputs=x)
    
    return model

In [25]:
from sklearn.model_selection import GroupKFold

gkf = GroupKFold(n_splits=5).split(X=df_train.q2, groups=df_train.id)

valid_preds = []
test_preds = []

oof = np.zeros((len(df_train),1))
for fold, (train_idx, valid_idx) in enumerate(gkf):
    train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
    train_outputs = outputs[train_idx]
    valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
    valid_outputs = outputs[valid_idx]

    K.clear_session()
    model = create_model()
    optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=[tf.keras.metrics.AUC()])
    model.fit(train_inputs, train_outputs, validation_data = (valid_inputs, valid_outputs), epochs=3, batch_size=64)
    oof_p = model.predict(valid_inputs, batch_size=512)
    oof[valid_idx] = oof_p
    valid_preds.append(oof_p)
    test_preds.append(model.predict(test_inputs, batch_size=512))
    f1,t = search_f1(valid_outputs, valid_preds[-1])
    print('validation score = ', f1)

I1015 10:23:27.365832 140024237856576 configuration_utils.py:283] loading configuration file ./bert-base-chinese-config.json
I1015 10:23:27.367100 140024237856576 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

I1015 10:23:27.367972 140024237856576 modeling_tf_utils.py:391] loading weights file ./bert-base-chinese-tf_model.h5
I1015 10:23:31.845176 140024237856576 model

Train on 17268 samples, validate on 4317 samples
Epoch 1/3


W1015 10:23:41.673791 140024237856576 optimizer_v2.py:1029] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.
W1015 10:23:51.115387 140024237856576 optimizer_v2.py:1029] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


17268/17268 [==============================] - 350s 20ms/sample - loss: 0.4541 - auc: 0.8335 - val_loss: 0.3003 - val_auc: 0.9264
Epoch 2/3
17268/17268 [==============================] - 324s 19ms/sample - loss: 0.3038 - auc: 0.9259 - val_loss: 0.2780 - val_auc: 0.9397
Epoch 3/3
17268/17268 [==============================] - 323s 19ms/sample - loss: 0.2387 - auc: 0.9528 - val_loss: 0.2845 - val_auc: 0.9396


I1015 10:44:54.028635 140024237856576 configuration_utils.py:283] loading configuration file ./bert-base-chinese-config.json
I1015 10:44:54.029900 140024237856576 configuration_utils.py:321] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

I1015 10:44:54.031107 140024237856576 modeling_tf_utils.py:391] loading weights file ./bert-base-chinese-tf_model.h5


...

Train on 17268 samples, validate on 4317 samples
Epoch 1/3


W1015 10:45:08.121540 140024237856576 optimizer_v2.py:1029] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.
W1015 10:45:17.625686 140024237856576 optimizer_v2.py:1029] Gradients do not exist for variables ['tf_bert_model/bert/pooler/dense/kernel:0', 'tf_bert_model/bert/pooler/dense/bias:0'] when minimizing the loss.


 9280/17268 [===============>..............] - ETA: 2:35 - loss: 0.5137 - auc: 0.7882

W1015 10:48:01.514178 140024237856576 callbacks.py:990] Can save best model only with val_loss available, skipping.


KeyboardInterrupt: 

In [23]:
best_score, best_t = search_f1(outputs,oof)

best 0.7719488011537767
thres 0.4


In [21]:
sub = np.average(test_preds, axis=0) 
sub = sub > best_t
df_test['label'] = sub.astype(int)
df_test[['id','id_sub','label']].to_csv('submission_beike_{}.csv'.format(best_score),index=False, header=None,sep='\t')